In [1]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from importlib import reload
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("/home/kim2712/Desktop/research/conformal_tree/")
sys.path.append("/home/kim2712/Desktop/research/generative_quantile/generative_qunatile/brenier/")

In [10]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning)

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data
import argparse
import numpy as np
import scipy
from scipy.stats import norm
#our libs
import matplotlib.pyplot as plt
import seaborn as sns
import utils
from utils import truncated_normal
from gen_data import *
from torchvision import datasets, transforms, utils
from models import ConditionalConvexQuantile
from models import dual
DEVICE ="cpu"# torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)

In [11]:
class Synthetic(data.Dataset):
    def __init__(self, args,  n=1000, device=DEVICE):
        self.n = n
        np.random.seed(0)
        self.device = device
        #self.y = np.random.normal(loc=0, scale=1, size=(self.n, 1))
        #self.y = np.random.multivariate_normal(mean=[2, 3], cov=np.array([[3,2],[2,5]]), size=(self.n))

        #torch.manual_seed(0)
        self.y, self.x = make_spiral(n_samples_per_class=self.n, n_classes=3,
            n_rotations=1.5, gap_between_spiral=1.0, noise=0.2,
                gap_between_start_point=0.1, equal_interval=True)
        '''

        self.y, self.x = make_moons(n_samples=args.n, xy_ratio=2.0, x_gap=-0.2, y_gap=0.2, noise=0.1)
        '''

    def __len__(self):
        return len(self.y)#self.y

    def __getitem__(self, i):
        if torch.is_tensor(self.y):
            return self.y[i].float().to(self.device), self.x[i].to(self.device)
        y = torch.from_numpy(self.y[i]).float().to(self.device)
        x = torch.from_numpy(np.array(self.x[i])).to(self.device)
        return y, x

In [12]:
def plot2d(Y, name, labels=None):
    Y = Y.detach().cpu().numpy()
    #labels = labels.detach().cpu().numpy().flatten()
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1, 1, 1)
    #sns.kdeplot(Y[:, 0], Y[:, 1], cmap='Blues', shade=True, thresh=0)
    sns.scatterplot(x=Y[:,0], y=Y[:,1], hue=labels, s=5)
    '''
    H, _, _ = np.histogram2d(Y[:, 0], Y[:, 1], 200, range=[[-4, 4], [-4, 4]])
    plt.imshow(H.T, cmap='BuPu')
    '''
    ax.get_xaxis().set_ticks([])
    ax.get_yaxis().set_ticks([])
    plt.axis('off')
    plt.tight_layout()
    plt.savefig("./" + name)
    plt.clf()

def histogram(Y, name):
    Y = Y.detach().cpu().numpy()
    plt.hist(Y, bins=25)
    plt.savefig("./" + name)
    plt.clf()

def plotaxis(Y, name):
    y1, y2 = Y[:,0], Y[:,1]
    histogram(y1, name=str(name)+'_x1.png')
    histogram(y2, name=str(name)+'_x2.png')

def gaussian_mixture(means, stds, p, args):
    assert np.sum(p) == 1
    k = len(p)
    ranges = [0.]
    for i in range(k):
        ranges.append(ranges[i] + p[i])
    mix = np.zeros((args.n, 1))
    idx = np.random.uniform(0, 1, size=(args.n, 1))
    for i in range(k):
        g = np.random.normal(loc=means[i], scale=stds[i], size=(args.n, 1))
        indices = np.logical_and(idx >= ranges[i], idx < ranges[i+1])
        mix[indices] = g[indices]
    return mix

def Optimizer(net, args):
    assert args.optimizer.lower() in ["sgd", "adam", "radam"], "Invalid Optimizer"

    if args.optimizer.lower() == "sgd":
	       return optim.SGD(net.parameters(), lr=args.lr, momentum=args.beta1, nesterov=args.nesterov)
    elif args.optimizer.lower() == "adam":
	       return optim.Adam(net.parameters(), lr=args.lr, betas=(args.beta1, args.beta2))

def unif(size, eps=1E-7):
    return torch.clamp(torch.rand(size).to(DEVICE), min=eps, max=1-eps)

def test(net, args, name, loader):
    net.eval()

    '''
    for p in list(net.parameters()):
        if hasattr(p, 'be_positive'):
            print(p)
    '''
    gauss = torch.distributions.normal.Normal(torch.tensor([0.]).to(DEVICE), torch.tensor([1.]).to(DEVICE))
    U = unif(size=(5000, 2))
    U = gauss.icdf(U)
    X = torch.zeros(5000, device=device).long()
    X[:5000//3] = 1
    X[5000//3: 10000//3] = 2
    #print(X)
    Y_hat = net.grad(U, X)#= net.forward(U, grad=True).sum()
    #Y_hat = net.grad(U)
    print("max and min points generated: " + str(Y_hat.max()) + " " + str(Y_hat.min()))
    '''

    inverse = net.invert(Y_hat)
    m = (U - inverse).abs().max().item()
    print("max error of inversion: " + str(m))
    data = torch.sort(Y, dim=0)[0]
    z = net.invert(data)
    z = gauss.cdf(z)
    print("sampled points from target, sorted: " + str(data))
    print("corresponding quantiles: " + str(z))
    '''
    print(Y_hat.shape)
    plot2d(Y_hat, name='imgs/2d.png', labels=X.cpu().numpy()) # 2d contour plot
    #plotaxis(Y_hat, name='imgs/train')

positive_params = []

In [13]:
x.permute(1, 0,2)

NameError: name 'x' is not defined

In [17]:
def train(net, optimizer, loader, args):
    k = args.k
    #eg = Rings() # EightGaussian()
    gauss = torch.distributions.normal.Normal(torch.tensor([0.]).to(DEVICE), torch.tensor([1.]).to(DEVICE))
    for epoch in range(1, args.epoch+1):
        running_loss = 0.0
        for idx, (Y, label) in enumerate(loader):
            #Y = eg.sample(args.batch_size).cuda()
            u = unif(size=(args.batch_size, 2))#args.dims))
            u = gauss.icdf(u)
            optimizer.zero_grad()
            #label[:] = 0
            #label[:args.batch_size//2] = 1
            X,_ = net.to_onehot(label, 3)
            alpha, beta= net(u)
            loss = dual(U=u, Y_hat=(alpha, beta), Y=Y, X=X, eps=args.eps)
            loss.backward()
            optimizer.step()
            #for p in positive_params:
            #    p.data.copy_(torch.relu(p.data))
            running_loss += loss.item()
        if epoch % 5 == 0:
            print('%.5f' %
                (running_loss))#/(idx+1)))

    test(net, args, name='imgs/trained.png', loader=loader)
    '''
    Y = torch.tensor(loader.dataset.y)#eg.sample(1000).cuda()
    X = loader.dataset.x
    plot2d(Y, labels=X, name='imgs/theor.png')
    '''

In [18]:
class args:
    batch_size = 128 
    epoch = 10 
    optimizer='adam'
    lr=0.005
    beta1 = 0.9 
    beta2=0.999
    nesterov=False
    iters=1000
    mean=0
    std=1
    
    m=10
    n=5000
    k=100
    #genTheor=True
    #gaussian_support=True
    eps=0

In [19]:
print("Input arguments:")
for key, val in vars(args).items():
    print("{:16} {}".format(key, val))

#torch.cuda.set_device('cuda:0')
net = ConditionalConvexQuantile(xdim=3,ydim=2, 
                                a_hid=512,
                                a_layers=3,
                                b_hid=512,
                                b_layers=3)

#for p in list(net.parameters()):
#    if hasattr(p, 'be_positive'):
#        positive_params.append(p)
#    p.data = torch.from_numpy(truncated_normal(p.shape, threshold=1./np.sqrt(p.shape[1] if len(p.shape)>1 else p.shape[0]))).float()

ds = Synthetic(args, n=args.n)
loader = data.DataLoader(ds, batch_size=args.batch_size, shuffle=True, drop_last=True)
optimizer = Optimizer(net, args)
net.to(DEVICE)#.cuda()
train(net, optimizer, loader, args)
#mnist
#train(net, optimizer, loader, ds.y[:args.n].float().cuda(), args)

if args.genTheor:
    Y = torch.from_numpy(ds.y)
    #plotaxis(Y, name='imgs/theor')
    plot2d(Y, labels=ds.x, name='imgs/theor.png')

print("Training completed!")

Input arguments:
__module__       __main__
batch_size       128
epoch            10
optimizer        adam
lr               0.005
beta1            0.9
beta2            0.999
nesterov         False
iters            1000
mean             0
std              1
m                10
n                5000
k                100
eps              0
__dict__         <attribute '__dict__' of 'args' objects>
__weakref__      <attribute '__weakref__' of 'args' objects>
__doc__          None


ZeroDivisionError: integer division or modulo by zero

In [44]:
k = args.k
#eg = Rings() # EightGaussian()
gauss = torch.distributions.normal.Normal(torch.tensor([0.]).to(DEVICE), torch.tensor([1.]).to(DEVICE))
for epoch in range(1, args.epoch+1):
    running_loss = 0.0

In [45]:
gauss = torch.distributions.normal.Normal(torch.tensor([0.]).to(DEVICE), torch.tensor([1.]).to(DEVICE))

In [46]:
for idx, (Y, label) in enumerate(loader):
    #Y = eg.sample(args.batch_size).cuda()
    u = unif(size=(args.batch_size, 2))#args.dims))
    u = gauss.icdf(u)
    optimizer.zero_grad()
    #label[:] = 0
    #label[:args.batch_size//2] = 1
    X,_ = net.to_onehot(label,3)
    alpha, beta= net(u)
    loss = dual(U=u, Y_hat=(alpha, beta), Y=Y, X=X, eps=args.eps)
    loss.backward()
    optimizer.step()
    #for p in positive_params:
    #    p.data.copy_(torch.relu(p.data))
    running_loss += loss.item()

RuntimeError: input.size(-1) must be equal to input_size. Expected 2, got 3

In [33]:
net#net(u)

ConditionalConvexQuantile(
  (alpha): ICNN_LastInp_Quadratic(
    (fc1_normal): Linear(in_features=3, out_features=512, bias=True)
    (activ_1): CELU(alpha=1.0)
    (normal): ModuleList(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): Linear(in_features=3, out_features=512, bias=True)
    )
    (convex): ModuleList(
      (0): ConvexLinear(in_features=512, out_features=512, bias=False)
      (1): ConvexLinear(in_features=512, out_features=512, bias=False)
    )
    (activation): ModuleList(
      (0): CELU(alpha=1.0)
      (1): CELU(alpha=1.0)
    )
    (last_convex): ConvexLinear(in_features=512, out_features=1, bias=False)
    (last_linear): Linear(in_features=3, out_features=1, bias=True)
  )
  (beta): ICNN_LastInp_Quadratic(
    (fc1_normal): Linear(in_features=3, out_features=512, bias=True)
    (activ_1): CELU(alpha=1.0)
    (normal): ModuleList(
      (0): Linear(in_features=3, out_features=512, bias=True)
      (1): Linear(in_features=3, out_features=5

In [30]:
X

tensor([[ 0.8276,  0.6544,  0.0730],
        [-1.1130, -0.4055,  0.2691],
        [ 0.8276,  0.6544,  0.0730],
        [ 0.8276,  0.6544,  0.0730],
        [-1.1130, -0.4055,  0.2691],
        [ 0.8276,  0.6544,  0.0730],
        [ 0.8276,  0.6544,  0.0730],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.8276,  0.6544,  0.0730],
        [-1.1130, -0.4055,  0.2691],
        [ 0.3165, -0.2136, -0.3309],
        [-1.1130, -0.4055,  0.2691],
        [ 0.3165, -0.2136, -0.3309],
        [-1.1130, -0.4055,  0.2691],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.8276,  0.6544,  0.0730],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.8276,  0.6544,  0.0730],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.3165, -0.2136, -0.3309],
        [ 0.8276,  0.6544,  0.0730],
        [-1.1130, -0.4055,  0.2691],
 

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x2 and 3x512)

In [26]:
X = net.to_onehot(label,3)

RuntimeError: input must have 3 dimensions, got 2

In [27]:
label.shape

torch.Size([128])

In [28]:
self = net

In [29]:
onehot = torch.zeros((label.shape[0], 3), device="cpu")

In [30]:
label.reshape(-1, 1).shape,self.xdim

(torch.Size([128, 1]), 3)

In [31]:
onehot.scatter_(dim=-1, index=label.reshape(-1, 1), value=1)

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1

In [32]:
n_c=3
onehot = torch.zeros((label.shape[0], n_c), device="cpu")

In [33]:
onehot.scatter_(dim=-1, index=label.view(label.shape[0], 1), value=1)

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1

In [34]:
onehot

tensor([[0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1

In [35]:
self.f(onehot)

RuntimeError: input must have 3 dimensions, got 2

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from ot_modules.icnn import *

In [37]:
device="cpu"
class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, xdim, bn_last=True):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bn_last = bn_last
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=False)
        self.fc = nn.Linear(hidden_size, xdim)
        self.norm = nn.BatchNorm1d(xdim, momentum=1.0, affine=False)
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        if self.bn_last:
            return self.norm(out), out
        return out


In [38]:
f = BiRNN(input_size=3,
                       hidden_size=512,#args.dims*4,
                       num_layers=1,
                       xdim=3)

In [39]:
self=f
x = onehot

In [40]:
h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device) # 2 for bidirection 

In [41]:
c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
# Forward propagate LSTM

In [43]:
out, _ = self.lstm(x.view(1,128,3))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)

In [44]:
x.shape, h0.shape, c0.shape

(torch.Size([128, 3]), torch.Size([1, 128, 512]), torch.Size([1, 128, 512]))

In [45]:
# Decode the hidden state of the last time step
out = self.fc(out[:, -1, :])
if self.bn_last:
    return self.norm(out), out
return out

SyntaxError: 'return' outside function (<ipython-input-45-dfe44f778231>, line 4)

In [31]:
f(onehot)

RuntimeError: input must have 3 dimensions, got 2

In [31]:
self.f

BiRNN(
  (lstm): LSTM(2, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=3, bias=True)
  (norm): BatchNorm1d(3, eps=1e-05, momentum=1.0, affine=False, track_running_stats=True)
)

In [32]:
self.f

BiRNN(
  (lstm): LSTM(2, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=3, bias=True)
  (norm): BatchNorm1d(3, eps=1e-05, momentum=1.0, affine=False, track_running_stats=True)
)

In [18]:
label

tensor([0, 2, 0, 1, 1, 2, 0, 2, 1, 0, 2, 0, 2, 2, 0, 2, 1, 0, 1, 1, 2, 1, 2, 0,
        0, 2, 2, 0, 1, 2, 2, 2, 0, 1, 1, 0, 0, 1, 1, 1, 1, 2, 1, 0, 1, 2, 2, 2,
        1, 1, 2, 1, 1, 1, 2, 2, 0, 2, 0, 2, 1, 1, 1, 2, 2, 2, 0, 1, 2, 0, 0, 2,
        2, 2, 2, 1, 1, 0, 0, 0, 1, 1, 2, 2, 1, 1, 2, 0, 1, 1, 1, 0, 1, 0, 1, 2,
        0, 1, 1, 2, 0, 2, 2, 1, 0, 0, 0, 1, 0, 1, 2, 0, 1, 2, 2, 1, 1, 0, 2, 2,
        2, 1, 1, 2, 1, 2, 1, 1])

In [19]:
net.to_onehot

<bound method ConditionalConvexQuantile.to_onehot of ConditionalConvexQuantile(
  (alpha): ICNN_LastInp_Quadratic(
    (fc1_normal): Linear(in_features=2, out_features=512, bias=True)
    (activ_1): CELU(alpha=1.0)
    (normal): ModuleList(
      (0): Linear(in_features=2, out_features=512, bias=True)
      (1): Linear(in_features=2, out_features=512, bias=True)
    )
    (convex): ModuleList(
      (0): ConvexLinear(in_features=512, out_features=512, bias=False)
      (1): ConvexLinear(in_features=512, out_features=512, bias=False)
    )
    (activation): ModuleList(
      (0): CELU(alpha=1.0)
      (1): CELU(alpha=1.0)
    )
    (last_convex): ConvexLinear(in_features=512, out_features=1, bias=False)
    (last_linear): Linear(in_features=2, out_features=1, bias=True)
  )
  (beta): ICNN_LastInp_Quadratic(
    (fc1_normal): Linear(in_features=2, out_features=512, bias=True)
    (activ_1): CELU(alpha=1.0)
    (normal): ModuleList(
      (0): Linear(in_features=2, out_features=512, bias=

In [ ]:
Y = torch.from_numpy(ds.y)
#plotaxis(Y, name='imgs/theor')
plot2d(Y, labels=ds.x, name='imgs/theor.png')

In [16]:
ds.x, ds.y

(array([0, 0, 0, ..., 2, 2, 2]),
 array([[10.50609033,  9.03991174],
        [-1.93926701, 15.6387472 ],
        [ 7.54439633, 12.85390011],
        ...,
        [ 3.37052028,  5.15696014],
        [16.84918769,  2.74954539],
        [13.72389705, 12.08289123]]))